# ArcGIS Portal Dependencies
**Description**: This script pulls data for all items in an organization's ArcGIS Portal and creates a list of dependencies, which can both be exported CSV files to be used for data management.

## Parameters
**Login**

`portal_url`: URL to your organization's ArcGIS Portal. Set this equal to `"pro"` to use ArcGIS Pro's login method. Must include "https://" if a URL is used.

`gis_user`: Username used to login to your organization. Leave this blank if `portal_url` is `"pro"`.

`gis_pass`: Password used to login to your organization. Leave this blank if `portal_url` is `"pro"`.

---

**Search/Filters**

`filter_type`: Filter method to use for items.
* Options: `""` (no filter), `"Exclude"` (remove any items with a type in `type_filter`), `"Include"` (keep any items with a type in `type_filter`).

`type_filter`: Types to use with specified filter method. The constant `PORTAL_ITEM_TYPES` lists all possible types to filter on.

`drop_columns`: Data to drop from item data. The constant `ARC_ITEM_FIELDS` lists the possible item data fields.

`item_rename_columns`: Column names to use in place of the default ones for item data. This is a dictionary with keys representing the old name, and values representing the new name. The constant `ARC_ITEM_FIELDS` lists the possible default item data fields. Three new columns are generated in this script and can also be renamed: `"totalDependentTo"`, `"totalDependsUpon"`, and `"group"`.

`depends_rename_columns`: Column names to use in place of the default ones for dependencies data. This is a dictionary with keys representing the old name, and values representing the new name. The following are default field names: `"sourceId"`, `"sourceTitle"`, `"sourceType"`, `"dependency"`, `"dependencyType"`, `"dependencyTitle"`, `"dependencyItemType"`, `"broken"`.

---

**Configs**

`exclude_serverid`: Whether or not to exclude serverId dependency types in the final dataset. Removing them here also prevents them from affecting dependency counts.

`csv_folder`: Folder to save CSV files in.

`export_name_items`: Name to use when creating tables for item data.

`export_name_depends`: Name to use when creating tables for dependencies data.

In [8]:
#Parameters
##Login
portal_url = 'pro'
gis_user = ''
gis_pass = ''

##Search/Filter
filter_type = ''
type_filter = []
drop_columns = ['isOrgItem','guid','name','typeKeywords','thumbnail','documentation','extent','spatialReference',
                'accessInformation','licenseInfo','culture','properties','advancedSettings','proxyFilter',
                'subInfo','appCategories','industries','languages','largeThumbnail','banner','screenshots',
                'listed','ownerFolder','protected','numComments','numRatings','avgRating','scoreCompleteness', 
                'groupDesignations','lastViewed','layers','tables','origin','sourceUrl']

item_rename_columns = {'group':'group_','totalDependsUpon':'totalDependencies','totalDependentTo':'totalDepends'}
depends_rename_columns = {}

#Configuration
exclude_serverid = True

##Files
csv_folder = 'C:/path/to/folder'
export_name_items = 'Portal_Items'
export_name_depends = 'Portal_Dependencies'

###  Parameter Validation

In [9]:
#Constants
"""A list of item types present in ArcGIS Portal. Derived from: https://developers.arcgis.com/rest/users-groups-and-items/items-and-item-types/."""
PORTAL_ITEM_TYPES = ['360 VR Experience','CityEngine Web Scene','Map Area','Pro Map','Web Map','Web Scene','Feature Collection','Feature Collection Template','Feature Service','Geodata Service','Group Layer','Image Service','KML','KML Collection','Map Service','OGCFeatureServer','Oriented Imagery Catalog','Relational Database Connection','3DTilesService','Scene Service','Vector Tile Service','WFS','WMS','WMTS','Geometry Service','Geocoding Service','Geoprocessing Service','Network Analysis Service','Workflow Manager Service','AppBuilder Extension','AppBuilder Widget Package','Code Attachment','Dashboard','Data Pipeline','Deep Learning Studio Project','Esri Classification Schema','Excalibur Imagery Project','Experience Builder Widget','Experience Builder Widget Package','Form','GeoBIM Application','GeoBIM Project','Hub Event','Hub Initiative','Hub Initiative Template','Hub Page','Hub Project','Hub Site Application','Insights Workbook','Insights Workbook Package','Insights Model','Insights Page','Insights Theme','Insights Data Engineering Workbook','Insights Data Engineering Model','Investigation','Knowledge Studio Project','Mission','Mobile Application','Notebook','Notebook Code Snippet Library','Native Application','Native Application Installer','Ortho Mapping Project','Ortho Mapping Template','Solution','StoryMap','Web AppBuilder Widget','Web Experience','Web Experience Template','Web Mapping Application','Workforce Project','Administrative Report','Apache Parquet','CAD Drawing','Color Set','Content Category Set','CSV','Document Link','Earth configuration','Esri Classifier Definition','Export Package','File Geodatabase','GeoJson','GeoPackage','GML','Image','iWork Keynote','iWork Numbers','iWork Pages','Microsoft Excel','Microsoft Powerpoint','Microsoft Word','PDF','Report Template','Service Definition','Shapefile','SQLite Geodatabase','Statistical Data Collection','StoryMap Theme','Style','Symbol Set','Visio Document','ArcPad Package','Compact Tile Package','Explorer Map','Globe Document','Layout','Map Document','Map Package','Map Template','Mobile Basemap Package','Mobile Map Package','Mobile Scene Package','Project Package','Project Template','Published Map','Scene Document','Task File','Tile Package','Vector Tile Package','Explorer Layer','Image Collection','Layer','Layer Package','Pro Report','Scene Package','3DTilesPackage','Desktop Style','ArcGIS Pro Configuration','Deep Learning Package','Geoprocessing Package','Geoprocessing Package (Pro version)','Geoprocessing Sample','Locator Package','Raster function template','Rule Package','Pro Report Template','ArcGIS Pro Add In','Code Sample','Desktop Add In','Desktop Application','Desktop Application Template','Explorer Add In','Survey123 Add In','Workflow Manager Package']

"""A list of fields attached to ArcGIS item objects that contain item properties."""
ARC_ITEM_FIELDS = ['id', 'owner', 'created', 'isOrgItem', 'modified', 'guid', 'name', 'title', 'type', 'typeKeywords', 'description', 'tags', 'snippet', 'thumbnail', 'documentation', 'extent', 'categories', 'spatialReference', 'accessInformation', 'licenseInfo', 'culture', 'properties', 'advancedSettings', 'url', 'proxyFilter', 'access', 'subInfo', 'appCategories', 'industries', 'languages', 'largeThumbnail', 'banner', 'screenshots', 'listed', 'ownerFolder', 'protected', 'numComments', 'numRatings', 'avgRating', 'numViews', 'scoreCompleteness', 'groupDesignations', 'lastViewed', 'layers', 'tables', 'origin', 'sourceUrl']

"""A list of typical data types for ArcGIS item object fields that contain item properties (extension of `ARC_ITEM_FIELDS`."""
ARC_ITEM_FIELD_DTYPES = {'id':str, 'owner':str, 'created':str, 'isOrgItem':bool, 'modified':str, 'guid':str, 'name':str, 'title':str, 'type':str, 'typeKeywords':list, 'description':str, 'tags':list, 'snippet':str, 'thumbnail':str, 'documentation':str, 'extent':list, 'categories':list, 'spatialReference':str, 'accessInformation':str, 'culture':str, 'properties':dict, 'advancedSettings':'???', 'url':str, 'proxyFilter':'???', 'access':str, 'subInfo':int, 'appCategories':list, 'industries':list, 'languages':list, 'largeThumbnail':str, 'banner':str, 'screenshots':list, 'listed':bool, 'ownerFolder':str, 'protected':bool, 'numComments':int, 'numRatings':int, 'avgRating':float, 'numViews':int, 'scoreCompleteness':int, 'groupDesignations':'???', 'lastViewed':int, 'layers':list,'tables':list,'origin':str,'sourceUrl':str}

#Validation
if filter_type not in ['','Include','Exclude']:
    raise ValueError('filter_type is not a valid value.')
    
for i in drop_columns:
    if i not in ARC_ITEM_FIELDS:
         raise ValueError(f'"drop_columns" contains a value that is not a valid column name: {i}.')
        
for i in item_rename_columns:
    if i not in ARC_ITEM_FIELDS and i not in ["totalDependentTo", "totalDependsUpon", "group"]:
        raise ValueError(f'"item_rename_columns" contains a value that is not a valid column name: {i}.')

for i in depends_rename_columns:
    if i not in ['sourceId', 'sourceTitle', 'sourceType', 'dependency', 'dependencyTitle', 'dependencyItemType', 'broken']:
        raise ValueError(f'"depends_rename_columns" contains a value that is not a valid column name: {i}.')

for i in drop_columns:
    if i in item_rename_columns or i in depends_rename_columns:
        raise ValueError(f'The field "{i}" is set as a field to both drop and rename.')

if not isinstance(exclude_serverid, bool):
    raise ValueError('exclude_serverid must be a boolean value.')
if not isinstance(upload, bool):
    raise ValueError('upload must be a boolean value.')

#Fieldmaps
if workspace[-1] != '/' and '/' in workspace:
    workspace = workspace + '/'
elif workspace[-1] != '\\' and '\\' in workspace:
    workspace = workspace + '\\'

if '<ENV/TBLLYR>' in item_field_map:
    item_field_map = item_field_map.replace('<ENV/TBLLYR>', f'{workspace}{export_name_items}')

if '<ENV/TBLLYR>' in depends_field_map:
    depends_field_map = item_field_map.replace('<ENV/TBLLYR>', f'{workspace}{export_name_depends}')

## Import Modules

In [5]:
#Import Modules
import warnings
import requests
from datetime import datetime
from arcgis import GIS
from pandas import DataFrame, to_datetime

## Defined Functions

In [6]:
"""A list of item types present in ArcGIS Portal. Derived from: https://developers.arcgis.com/rest/users-groups-and-items/items-and-item-types/."""
PORTAL_ITEM_TYPES = ['360 VR Experience','CityEngine Web Scene','Map Area','Pro Map','Web Map','Web Scene','Feature Collection','Feature Collection Template','Feature Service','Geodata Service','Group Layer','Image Service','KML','KML Collection','Map Service','OGCFeatureServer','Oriented Imagery Catalog','Relational Database Connection','3DTilesService','Scene Service','Vector Tile Service','WFS','WMS','WMTS','Geometry Service','Geocoding Service','Geoprocessing Service','Network Analysis Service','Workflow Manager Service','AppBuilder Extension','AppBuilder Widget Package','Code Attachment','Dashboard','Data Pipeline','Deep Learning Studio Project','Esri Classification Schema','Excalibur Imagery Project','Experience Builder Widget','Experience Builder Widget Package','Form','GeoBIM Application','GeoBIM Project','Hub Event','Hub Initiative','Hub Initiative Template','Hub Page','Hub Project','Hub Site Application','Insights Workbook','Insights Workbook Package','Insights Model','Insights Page','Insights Theme','Insights Data Engineering Workbook','Insights Data Engineering Model','Investigation','Knowledge Studio Project','Mission','Mobile Application','Notebook','Notebook Code Snippet Library','Native Application','Native Application Installer','Ortho Mapping Project','Ortho Mapping Template','Solution','StoryMap','Web AppBuilder Widget','Web Experience','Web Experience Template','Web Mapping Application','Workforce Project','Administrative Report','Apache Parquet','CAD Drawing','Color Set','Content Category Set','CSV','Document Link','Earth configuration','Esri Classifier Definition','Export Package','File Geodatabase','GeoJson','GeoPackage','GML','Image','iWork Keynote','iWork Numbers','iWork Pages','Microsoft Excel','Microsoft Powerpoint','Microsoft Word','PDF','Report Template','Service Definition','Shapefile','SQLite Geodatabase','Statistical Data Collection','StoryMap Theme','Style','Symbol Set','Visio Document','ArcPad Package','Compact Tile Package','Explorer Map','Globe Document','Layout','Map Document','Map Package','Map Template','Mobile Basemap Package','Mobile Map Package','Mobile Scene Package','Project Package','Project Template','Published Map','Scene Document','Task File','Tile Package','Vector Tile Package','Explorer Layer','Image Collection','Layer','Layer Package','Pro Report','Scene Package','3DTilesPackage','Desktop Style','ArcGIS Pro Configuration','Deep Learning Package','Geoprocessing Package','Geoprocessing Package (Pro version)','Geoprocessing Sample','Locator Package','Raster function template','Rule Package','Pro Report Template','ArcGIS Pro Add In','Code Sample','Desktop Add In','Desktop Application','Desktop Application Template','Explorer Add In','Survey123 Add In','Workflow Manager Package']

"""A list of fields attached to ArcGIS item objects that contain item properties."""
ARC_ITEM_FIELDS = ['id', 'owner', 'created', 'isOrgItem', 'modified', 'guid', 'name', 'title', 'type', 'typeKeywords', 'description', 'tags', 'snippet', 'thumbnail', 'documentation', 'extent', 'categories', 'spatialReference', 'accessInformation', 'licenseInfo', 'culture', 'properties', 'advancedSettings', 'url', 'proxyFilter', 'access', 'subInfo', 'appCategories', 'industries', 'languages', 'largeThumbnail', 'banner', 'screenshots', 'listed', 'ownerFolder', 'protected', 'numComments', 'numRatings', 'avgRating', 'numViews', 'scoreCompleteness', 'groupDesignations', 'lastViewed', 'layers', 'tables', 'origin', 'sourceUrl']

"""A list of typical data types for ArcGIS item object fields that contain item properties (extension of `ARC_ITEM_FIELDS`."""
ARC_ITEM_FIELD_DTYPES = {'id':str, 'owner':str, 'created':str, 'isOrgItem':bool, 'modified':str, 'guid':str, 'name':str, 'title':str, 'type':str, 'typeKeywords':list, 'description':str, 'tags':list, 'snippet':str, 'thumbnail':str, 'documentation':str, 'extent':list, 'categories':list, 'spatialReference':str, 'accessInformation':str, 'culture':str, 'properties':dict, 'advancedSettings':'???', 'url':str, 'proxyFilter':'???', 'access':str, 'subInfo':int, 'appCategories':list, 'industries':list, 'languages':list, 'largeThumbnail':str, 'banner':str, 'screenshots':list, 'listed':bool, 'ownerFolder':str, 'protected':bool, 'numComments':int, 'numRatings':int, 'avgRating':float, 'numViews':int, 'scoreCompleteness':int, 'groupDesignations':'???', 'lastViewed':int, 'layers':list,'tables':list,'origin':str,'sourceUrl':str}

def query_by_itemtype(filter_type: str = '', type_filter: list = None, gis = 'gis') -> list:
    '''
    An alternative to `gis.content.search()` that can retrieve data on more than 500 items and filter by item type. `gis.content.advanced_search()` could also be used, but has its own problems.
    `gis`: GIS login object.
        Default: `gis` (global variable)
    `filter_type`: Type of filter to apply for item types. `'Exclude'` to remove listed item types, '`Include`' to only keep the listed item types.
        Accepted Values: `'Exclude'`, `'Include'`, `''`
    '''
    #Get GIS Object
    if gis == 'gis':
        gis = globals()['gis']
    
    #Filter Types
    if filter_type == 'Exclude':
        type_filter = [i for i in PORTAL_ITEM_TYPES if i not in type_filter]
    elif filter_type == 'Include':
        type_filter = [i for i in PORTAL_ITEM_TYPES if i in type_filter]
    elif filter_type == '' or type_filter is None:
        type_filter = PORTAL_ITEM_TYPES
    else:
        raise ValueError(f'Correct filter type not specified. Exclude, Include, or an empty string are the only accepted inputs. Your input: {filter_type}.')
    
    #Build Query
    if isinstance(type_filter, list):
        query = '" OR "'.join(type_filter)
    else:
        query = type_filter
    query = f'type: ("{query}")'
    items = gis.content.search(query=f'{query} NOT owner: esri*', max_items=500)

    #Search by User, then by User:Itemtype if Max Items Returned
    if len(items) == 500:
        items = []
        users = gis.users.search(max_users = 10000)
        users = [user for user in users if user.storageUsage != 0]
        for user in users:
            items_by_user = gis.content.search(query=f'{query} owner: {user.username}', outside_org=False, max_items=500)
            if len(items_by_user) == 0:
                pass
            elif len(items_by_user) < 500:
                items.extend(items_by_user)
            elif len(items_by_user) == 500:
                for item_type in type_filter:
                    items_by_type = gis.content.search(query=f'type: {item_type} AND owner: {user.username}', outside_org=False, max_items=500)
                    if len(items_by_type) == 0:
                        pass
                    elif len(items_by_type) == 500:
                        warnings.warn(f'{user.username} has over 500 portal items of a single type. Unable to query all {item_type} items. Items currently queried added to content list.')
                        items.extend(items_by_type)
                    else:
                        items.extend(items_by_type)
    elif len(items) == 0:
        raise Exception('No items returned with current arguements.')
    
    #Fix Duplicates from Esri's Weird Query Method **eye roll**
    check_list = []
    for index, item in reversed(list(enumerate(items))):
        if item.id in check_list or item.type not in type_filter:
            items.pop(index)
        else:
            check_list.append(item.id)

    return items

def get_dependencies(items: list, dependency_info: bool = False, null_value: any = None, token = None, gis = 'gis') -> list:
    """
    Gets dependencies for a list of ArcGIS items and converts it to a flat table 
    with additional information (type, title, broken status).
    
    Params:
    * `gis`: GIS login object.
        * Default: `gis` (global variable)
    * `items`: List of ArcGIS items as the arcgis.gis.item class.
    * `dependency_info`: Whether to get data for dependency item type, dependency title, and broken status.
        * Default: `False`
    * `null_value`: The value used when no data can be retrieved. Only applicable if `dependency_info` is `True`.
        * Default: `None`
    * `token`: Token string to use when accessing rest services. This creates a bearer token as a header.
        * Default: `None`
        
    Issues:
    - Not all item titles/types can be retrieved due to the variety of JSON schemas (and resulting dict keys) used. 
    Adding more values to `title_try` and `type_try` could help.
    """
    
    #Get GIS Object
    if gis == 'gis':
        gis = globals['gis']
    if token is not None:
        token = {'Authorization': f'Bearer {token}'}
        
    
    #Get Dependencies Data
    dependencies = []
    for item in items:
        dependency = item.dependent_upon()['list']
        if dependency != []:
            for depend in dependency:
                depend_type = list(depend.keys())[1]
                dependencies.append({'sourceId':item.id,'sourceTitle':item.title,'sourceType':item.type, 'dependency':depend[depend_type],'dependencyType':depend['dependencyType']})

    #Skip Next Section if Not Needed
    if dependency_info is False:
        return dependencies
    
    #Test Broken Links  
    check_dict = {'http://utility.arcgisonline.com/arcgis/rest/services/Geometry/GeometryServer':['Esri Geometry Server', 'Geometry Server', False]} #Format: {id: [dependencyTitle, dependencyItemType, broken]}
    for item in dependencies:
        #Check if Dependency has Already been Tested
        if item['dependency'] in check_dict:
            item_title = check_dict[item['dependency']][0]
            item_type = check_dict[item['dependency']][1]
            broken = check_dict[item['dependency']][2]
        #Check 
        elif item['dependencyType'] == 'id' and '/' not in item['dependency']:
            try:
                get_item = gis.content.get(item['dependency'])
                if get_item is not None:
                    item_title = get_item.title
                    item_type = get_item.type
                    broken = False
                else:
                    item_title = null_value
                    item_type = null_value
                    broken = True
            except:
                item_title = null_value
                item_type = null_value
                broken = True
        elif item['dependencyType'] in ['url','id']:
            url = item['dependency'] + '?f=json'
            try:
                #Request JSON Data
                if url == gis._portal.url + '?f=json':
                    response = requests.get(url, timeout = 15, headers = token)
                else:
                    response = requests.get(url, timeout = 15)
                if response.status_code == 200:
                    broken = False
                    item_json = response.json()
                    title_try = ['title','Title','name','Name','mapName']
                    type_try = ['type','Type','Item Type','item type']
                    
                    #Try Possible Keys for Title and Type
                    for title in title_try:
                        try:
                            item_title = item_json[title]
                            break
                        except:
                            item_title = null_value
                    for type_ in type_try:
                        try:
                            item_type = item_json[type_]
                            break
                        except:
                            item_type = null_value
                else:
                    item_title = null_value
                    item_type = null_value
                    broken = True
            except:
                item_title = null_value
                item_type = null_value
                broken = True
                
        #Handle Other Types (i.e. serverId)
        else:
            item_title = null_value
            item_type = null_value
            broken = null_value
        
        #Add Info to Dependencies
        item['dependencyTitle'] = item_title
        item['dependencyItemType'] = item_type
        item['broken'] = broken
        
        #Add to Check List if Needed
        if item['dependency'] not in check_dict:
            check_dict[item['dependency']] = [item_title, item_type, broken]

    return dependencies

## Login and Get Data

In [10]:
#Login to ArcGIS Portal
if portal_url.lower() == 'pro':
    gis = GIS(portal_url)
else:
    gis = GIS(portal_url, gis_user, gis_pass)

#Get Items & Dependencies
item_data = query_by_itemtype(gis = gis, filter_type = filter_type, type_filter = type_filter)
dependencies = get_dependencies(gis = gis, items = item_data, dependency_info = True, token = gis._con.token)

#Remove Server ID Dependencies
if exclude_serverid is True:
    for index, item in reversed(list(enumerate(dependencies))):
        if item['dependencyType'] == 'serverId':
            dependencies.pop(index)

#Calculate Total Dependencies
total_dependencies = {}
id_list = [item['sourceId'] for item in dependencies]
for val in id_list:
    total_dependencies[val] = total_dependencies.get(val, 0) + 1

total_depend_to = {}
id_list = [item['dependency'] for item in dependencies]
for val in id_list:
    total_depend_to[val] = total_depend_to.get(val, 0) + 1

#Get Group Data
groups_data = {item.id:group.title for group in gis.groups.search(max_groups=-1) for item in group.content()}
groups = {}
for item in groups_data:
    if item not in groups:
        groups[item] = groups_data[item]
    else:
        groups[item].append(groups_data[item])

## Clean Up/Reformat Data

In [11]:
#Clean Up/Format Data and Add New Columns
access_values = {'public':'Public','private':'Private','shared':'Shared with Other Users','org':'Shared with Organization'}
status_values = {'org_authoritative':'Authoritative','deprecated':'Deprecated'}

items = []
for item in item_data:
    item_info = {}
    #Deal With Field Value Formatting
    for key in item.keys():
        if key in drop_columns:
            continue

        #Deal with Lists
        elif key in ['typeKeywords', 'tags', 'extent', 'categories', 'appCategories', 'industries', 'languages', 'screenshots']:
            if len(item[key]) == 0 or item[key] == ['']:
                item_info[key] = None
            else:
                item_info[key] = item[key]

        #Swap access Values
        elif key == 'access':
            if item[key] in  access_values:
                item_info[key] = access_values[item[key]]
            else:
                item_info[key] = None

        #Swap contentStatus Values
        elif key == 'contentStatus':
            if item[key] in status_values:
                item_info[key] = status_values[item[key]]
            else:
                item_info[key] = None

        #All Other Fields
        else:
            item_info[key] = item[key]
    
    #Check if contentStatus has a Value 
    try:
        item_info['contentStatus']
    except:
        item_info['contentStatus'] = None

    #Add Groups, Dependencies Counts, and URL
    try:
        item_info['group'] = groups[item.id]
    except:
        item_info['group'] = None
    try:
        item_info['totalDependsUpon'] = total_dependencies[item.id]
    except:
        item_info['totalDependsUpon'] = 0
    try:
        item_info['totalDependentTo'] = total_depend_to[item.id]
    except:
        item_info['totalDependentTo'] = 0
        
    #Append
    item_info['url'] = portal_url + '/home/item.html?id=' + item.id
    items.append(item_info) 

#Convert to DataFrames
item_df = DataFrame(items)
dependencies_df = DataFrame(dependencies)

#Fix Datatypes
item_dtypes = {field:ARC_ITEM_FIELD_DTYPES[field] for field in ARC_ITEM_FIELD_DTYPES if field not in drop_columns and field not in ['created','modified']}
item_dtypes.update({'group':list, 'totalDependsUpon':int, 'totalDependentTo':int})
for i in item_dtypes:
    if item_dtypes[i] in [list,bool,dict,'???']:
        item_dtypes[i] = str
item_df = item_df.astype(item_dtypes, errors = 'ignore')
dependencies_df = dependencies_df.astype({'sourceId':str, 'sourceTitle': str, 'sourceType':str, 'dependency':str, 'dependencyTitle':str, 'dependencyItemType':str, 'broken':str}, errors = 'ignore')
item_df['created'] = to_datetime(item_df['created'],unit='ms').dt.date
item_df['modified'] = to_datetime(item_df['modified'],unit='ms').dt.date

#Rename Columns
if item_rename_columns != {}:
    item_df.rename(columns = item_rename_columns, inplace = True)
if depends_rename_columns != {}:
    dependencies_df.rename(columns = depends_rename_columns, inplace = True)

## Export

In [13]:

#Export as CSV
delim = '\\'
if csv_folder[-1] == '/':
    csv_folder = csv_folder.rstrip('/')
elif csv_folder[-1] == '\\':
    csv_folder = csv_folder.rstrip('\\')

item_df.to_csv(f'{csv_folder}{delim}{export_name_items}.csv', index = False)
dependencies_df.to_csv(f'{csv_folder}{delim}{export_name_depends}.csv', index = False)